In [50]:
from tools import *
from torch.autograd import Variable, Function
import torch
import math
import numpy as np
import torchvision
import pandas as pd
import torch.nn as nn
import nibabel as nib
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.utils.data as data_utils
import torch.optim.lr_scheduler as schd  
import torchvision.transforms as transforms
from sklearn.metrics import mean_absolute_error

%matplotlib inline

### Get Original Val

In [57]:
path = "/home/emy24/tadpole/data/"
datapath = path + "TADPOLE_TargetData_test.csv"
resampled, orig_dates = resample_data(datapath, interpolation = True)
orig_dates.iloc[781] = orig_dates.iloc[782]
resampled_val_pred = pd.read_csv("/home/emy24/tadpole/data/val_pred_resampled.csv")
resampled_val_pred = resampled_val_pred.set_index(resampled.index.values)
val = get_origDates(resampled_val_pred, orig_dates)
val_orig = pd.read_csv("/home/emy24/tadpole/data/TADPOLE_TargetData_test.csv")

v = get_origDates(resampled, orig_dates)

/nfs01/shared_software/anaconda3/envs/py35/lib/python3.5/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [102]:
np.sum(np.argmax(val.iloc[:,2:5].values, axis = 1) ==  np.argmax(v.iloc[:,2:5].values, axis = 1))/867

0.79700115340253752

In [95]:
val.to_csv("/home/emy24/tadpole/data/val_final_pred.csv")

### Test

In [91]:
path = "/home/emy24/tadpole/data/"
datapath = path + "TADPOLE_PredictTargetData_valid.csv"
resampled, orig_dates = resample_data(datapath, interpolation = True)
orig_dates.iloc[598] = orig_dates.iloc[599]
resampled_test_pred = pd.read_csv("/home/emy24/tadpole/data/test_pred_resampled.csv")
resampled_test_pred = resampled_test_pred.set_index(resampled.index.values)
test = get_origDates(resampled_test_pred, orig_dates)
test_orig = pd.read_csv("/home/emy24/tadpole/data/TADPOLE_PredictTargetData_valid.csv")

test.to_csv("/home/emy24/tadpole/data/test_final_pred.csv")

/nfs01/shared_software/anaconda3/envs/py35/lib/python3.5/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [94]:
test.to_csv("/home/emy24/tadpole/data/test_final_pred.csv")

### Load data

In [ ]:
path = "/home/zz452/tadpole/data/"
raw_data = pd.read_csv(path + "TADPOLE_TargetData_train.csv", low_memory=False)

# Convert date to date time
raw_data["Date"] = pd.to_datetime(raw_data["Date"])

# create two new columns to keep track if the datapoint is resampled, and the original date
raw_data["orig_date"] = 0

group = [data for _, data in raw_data.groupby("PTID_Key")] #Group the same patient into a list

### resampling

In [ ]:
resampled = []

for patient in group:

    patient.sort_values(by='Date') # sort the date by date
    num_data = patient.shape[0]
    time_diff = patient["Date"].diff()
    time_diff = time_diff / np.timedelta64(1,'M') #Convert to months
    num_resample = 1 # keep track of total number of resamples
    PTID_Key = patient.iloc[0,1]
    # resample the first data
    patient.iloc[0,-1] = patient.iloc[0,0]  # remember the orig date
    patient.iloc[0,0] = pd.to_datetime("2010-01-01") # resample date  
    
    for i in range(num_data-1): 
        
        resample_time = int( np.round(time_diff.iloc[i+1]/5) )

        # resample by (resample_time - 1) times        
        for j in range(resample_time-1):
            # add new points with NaNs except the date 
            new_date = pd.to_datetime("2010-01-01") + np.timedelta64(num_resample*5,'M')
            new = [new_date,PTID_Key,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]
            patient.loc[len(patient)] = new
            num_resample += 1
            
        # resample the date of the existing point 
        patient.iloc[i+1,-1] = patient.iloc[i+1,0]  # remember the orig date
        patient.iloc[i+1,0] = pd.to_datetime("2010-01-01") + np.timedelta64(num_resample*5,'M') # resample date
        num_resample += 1     
    
    # sort again after resampling
    output = patient.reset_index(drop=True)
    output = output.sort_values(by='Date') 
    
    # interpolations
    output.iloc[:,5:8] = output.iloc[:,5:8].interpolate(method = "linear").bfill()
    output.iloc[:,5:8] = output.iloc[:,5:8].interpolate(method = "linear").ffill()
    output.iloc[:,2:5] = output.iloc[:,2:5].bfill()
    output.iloc[:,2:5] = output.iloc[:,2:5].ffill()
    
    #If there is still missing value, fill it with the average of raw data
    output.iloc[:,5:8] = output.iloc[:,5:8].fillna(raw_data.iloc[:,5:8].mean())
    # For categorical data, NAN = 0
    output.iloc[:,2:5] = output.iloc[:,2:5].replace(np.NaN, 0)
    
    resampled.append(output)
    
resampled = pd.concat(resampled)
resampled = resampled.reset_index(drop=True)
            

### converting resampled data back to original format

In [ ]:
resampled.iloc[:,0] = orig_dates
resampled = resampled.dropna()

In [ ]:
get_origDates(resampled, orig_dates)

In [ ]:
resampled